In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
import os


In [2]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model

# --- Recreate your custom loss function exactly as used during training ---
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())  # numerical stability
        cross_entropy = -y_true * K.log(y_pred)
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy
        return K.sum(loss, axis=1)
    return focal_loss_fixed

# --- Now load the model with the custom loss registered ---
model = load_model(
    'densenet_focal_epoch2.h5',
    custom_objects={'focal_loss_fixed': focal_loss(gamma=2.0, alpha=0.25)}
)

print("✅ Model loaded successfully!")
model.summary()


✅ Model loaded successfully!


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 64, 64, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ densenet121 (Functional)        │ (None, 2, 2, 1024)     │     7,037,504 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1024)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       262,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1024)           │       263,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ classification (Dense)          │ (None, 14)             │         7,182 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,095,056 (30.88 MB)

 Trainable params: 8,011,406 (30.56 MB)

 Non-trainable params: 83,648 (326.75 KB)

 Optimizer params: 2 (12.00 B)

In [3]:
def predict_image(img_path, model, target_size=(64,64)):
    # Load and resize image
    img = image.load_img(img_path, target_size=target_size)
    
    # Convert to array
    img_array = image.img_to_array(img)
    
    # Expand dims to create batch of 1
    img_array = np.expand_dims(img_array, axis=0)
    
    # Normalize if your model was trained on normalized images
    img_array = img_array / 255.0  

    # Predict
    preds = model.predict(img_array)
    
    return preds


In [4]:
# Your original class mapping
classes = {
    'Abuse': 0, 'Arrest': 1, 'Arson': 2, 'Assault': 3,
    'Burglary': 4, 'Explosion': 5, 'Fighting': 6, 'NormalVideos': 7,
    'RoadAccidents': 8, 'Robbery': 9, 'Shooting': 10,
    'Shoplifting': 11, 'Stealing': 12, 'Vandalism': 13
}

# 🔄 Reverse the dictionary: index → class name
index_to_class = {v: k for k, v in classes.items()}

for i in range(11, 12):
    img_path = f'11.png'
    prediction = predict_image(img_path, model, target_size=(64,64))
    # print(f"Image: {img_path}, Prediction: {prediction}")
    predicted_class = np.argmax(prediction, axis=1)

    # Print result
    print("Predicted class index:", predicted_class[0])
    print("Predicted label:", index_to_class[predicted_class[0]])


# img_path = '10.png'
# prediction = predict_image(img_path, model, target_size=(64,64))

# print("Raw prediction:", prediction)


1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
Predicted class index: 7
Predicted label: NormalVideos


In [5]:
predicted_class = np.argmax(prediction, axis=1)

# Your original class mapping
classes = {
    'Abuse': 0, 'Arrest': 1, 'Arson': 2, 'Assault': 3,
    'Burglary': 4, 'Explosion': 5, 'Fighting': 6, 'NormalVideos': 7,
    'RoadAccidents': 8, 'Robbery': 9, 'Shooting': 10,
    'Shoplifting': 11, 'Stealing': 12, 'Vandalism': 13
}

# 🔄 Reverse the dictionary: index → class name
index_to_class = {v: k for k, v in classes.items()}

# Print result
print("Predicted class index:", predicted_class[0])
print("Predicted label:", index_to_class[predicted_class[0]])


Predicted class index: 7
Predicted label: NormalVideos
